In [ ]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

ws = Workspace.get(name='quick-starts-ws-136617',
                   subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860',
                   resource_group='aml-quickstarts-136617',
                   )

exp = Experiment(ws, 'myexperiment')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TOO Create compute cluster
cpu_cluster_name = "cpu-cluster"

# verify that the cluster is not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('cluster exist')
except ComputeTargetException:
    compute_gg = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_gg)

cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [ ]:
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.sklearn import SKLearn
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
      '--C': choice( 0.25, 0.5, 0.75, 1.0, 2),
      '--max_iter': choice(100, 150, 200, 250, 300)
  }
)


In [ ]:
# Specify policy
policy = BanditPolicy(
           slack_factor = 0.1,
           evaluation_interval = 2,
           delay_evaluation = 5)

In [ ]:
if "training" not in os.listdir():
    os.mkdir("./training")

# Creat a SKLearn estimator for the use with train.py
est = SKLearn(source_directory = './',
                     entry_script = './training/train.py',
                     compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy
hyperdrive_config = HyperDriveConfig(estimator = est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)


In [ ]:
# Submit your hyperparameter run to the experiment
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [ ]:
# show run details with the widgets
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

In [ ]:
parameter_values = best_run.get_details()['runDefinition']['arguments']

In [ ]:
print(parameter_values)

In [ ]:
best_model=best_run.register_model(model_name='Bestmodel.joblib',model_path='outputs')
best_model.id

In [ ]:
print('Best Run Id: ', best_run.id)